## Python Tkinter

#### Basic Window 

In [40]:
import tkinter as tk

root = tk.Tk()
root.title("My First Tkinter Window")
root.mainloop()


#### Adding Widgets

Here is an example of adding label and button to a Tkinter window

In [ ]:
import tkinter as tk
def on_button_click():
    print("Button clicked!")

root = tk.Tk()
root.title("Widgets Example")

# Adding a label
label = tk.Label(root, text="Hello, Tkinter!")
label.pack()

# Adding a button
button = tk.Button(root, text="Click Me", command=on_button_click)
button.pack()

root.mainloop()


#### Simple App

In [5]:
import tkinter as tk
def on_button_click():
    print("Button clicked!")

root = tk.Tk()
root.title("Widgets Example")


label = tk.Label(root, text="Enter your name:")
label.pack()

name_entry = tk.Entry(root)
name_entry.pack()


root.mainloop()


#### Simple App - using place()

In [6]:
import tkinter as tk

def on_button_click():
    print("Button clicked!")

root = tk.Tk()
root.title("Widgets Example")
root.geometry("300x200")  # Adjust the size as needed

# Placing the label
label = tk.Label(root, text="Enter your name:")
label.place(x=50, y=20)  # Adjust the position as needed

# Placing the entry field
name_entry = tk.Entry(root)
name_entry.place(x=50, y=50)  # Adjust the position as needed

root.mainloop()


#### Using Functions in Tkinter

In [12]:
import tkinter as tk

root = tk.Tk()
root.title("Greeting App")
root.geometry("300x150")  # Adjust the size as needed

def greet():
    name = name_entry.get()
    greeting_label.config(text=f"Hello, {name}!")

# Entry field for name
name_entry = tk.Entry(root)
name_entry.place(x=50, y=20)  # Adjust position as needed

# Button to trigger the greeting
greet_button = tk.Button(root, text="Greet", command=greet)
greet_button.place(x=50, y=50)  # Adjust position as needed

# Label to display the greeting
greeting_label = tk.Label(root, text="")
greeting_label.place(x=50, y=80)  # Adjust position as needed

root.mainloop()


#### Use Case: Building Text Editor

In [13]:
import tkinter as tk

root = tk.Tk()
root.title("Greeting and Text Editor App")
root.geometry("400x350")  # Adjust the window size as needed

# Function to display a greeting
def greet():
    name = name_entry.get()
    greeting_label.config(text=f"Hello, {name}!")

# Function to save text content
def save_text():
    text_content = text_editor.get("1.0", tk.END)
    with open("saved_text.txt", "w") as file:
        file.write(text_content)

# Entry field for name
name_entry = tk.Entry(root)
name_entry.place(x=50, y=20, width=200)

# Button to trigger the greeting
greet_button = tk.Button(root, text="Greet", command=greet)
greet_button.place(x=260, y=20)

# Label to display the greeting
greeting_label = tk.Label(root, text="")
greeting_label.place(x=50, y=50)

# Text widget for the text editor
text_editor = tk.Text(root, height=10)
text_editor.place(x=50, y=80, width=300, height=150)

# Button to save text
save_button = tk.Button(root, text="Save Text", command=save_text)
save_button.place(x=50, y=240)

root.mainloop()


#### Use Case:  Personal Finance Tracker

In [83]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
import time

# Initialize the main window
root = tk.Tk()
root.title("Personal Finance Tracker")
root.geometry("800x400")


# Database setup function
def create_database():
    conn = sqlite3.connect('finance_tracker.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS transactions
                      (id INTEGER PRIMARY KEY, amount REAL, category TEXT, date TEXT, description TEXT)''')
    conn.commit()
    conn.close()

editing_transaction_id = None

# Function to add a transaction
def add_transaction():
    global editing_transaction_id
    try:
        amount = float(amount_entry.get())
        category = category_combobox.get()
        description = description_entry.get()

        # Construct the date string
        day = day_combobox.get().zfill(2)  # Adds leading zero if needed
        month = str(month_combobox.current() + 1).zfill(2)  # Month as a number
        year = year_combobox.get()
        date = f"{year}-{month}-{day}"

        conn = sqlite3.connect('finance_tracker.db')
        cursor = conn.cursor()

        if editing_transaction_id is None:
            cursor.execute('''INSERT INTO transactions (amount, category, date, description)
                              VALUES (?, ?, ?, ?)''', (amount, category, date, description))
        else:
            cursor.execute('''UPDATE transactions SET amount = ?, category = ?, date = ?, description = ?
                              WHERE id = ?''', (amount, category, date, description, editing_transaction_id))
            editing_transaction_id = None

        conn.commit()
        conn.close()
        #clear_entries()

        # Call refresh_transactions() after commit and close operations
        refresh_transactions()

    except ValueError:
        messagebox.showerror("Error", "Invalid input. Please enter a valid number for the amount.")
    refresh_transactions()


    
def delete_transaction():
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showerror("Error", "No transaction selected")
        return

    transaction_id = tree.item(selected_item[0])['values'][0]
    response = messagebox.askyesno("Confirm Delete", "Are you sure you want to delete this transaction?")
    if response:
        conn = sqlite3.connect('finance_tracker.db')
        cursor = conn.cursor()
        cursor.execute("DELETE FROM transactions WHERE id = ?", (transaction_id,))
        conn.commit()
        conn.close()
        refresh_transactions()
        
def edit_transaction():
    global editing_transaction_id
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showerror("Error", "No transaction selected")
        return

    editing_transaction_id = tree.item(selected_item[0])['values'][0]
    conn = sqlite3.connect('finance_tracker.db')
    cursor = conn.cursor()
    cursor.execute("SELECT amount, category, description FROM transactions WHERE id = ?", (editing_transaction_id,))
    record = cursor.fetchone()
    conn.close()

    if record:
        amount_entry.delete(0, tk.END)
        amount_entry.insert(0, record[0])
        category_combobox.set(record[1])  # Set the value for the combobox
        description_entry.delete(0, tk.END)
        description_entry.insert(0, record[2])
        
# Function to refresh the transactions displayed
def refresh_transactions():
    total_expenses = 0
    total_savings = 0

    # Clear existing rows in the Treeview
    for row in tree.get_children():
        tree.delete(row)

    # Connect to the database and fetch new transaction data
    conn = sqlite3.connect('finance_tracker.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM transactions")
    rows = cursor.fetchall()
    conn.close()

    # Check if 'All' is selected or specific month/year
    selected_month = filter_month_combobox.get()
    selected_year = filter_year_combobox.get()

    # Convert selected month to its corresponding number
    if selected_month != "All":
        selected_month = str(month_combobox["values"].index(selected_month) + 1).zfill(2)

    # Filter the rows based on the selected month and year
    filtered_rows = []
    for row in rows:
        transaction_date = row[3]  # Assuming the date is in the 4th column (index 3)
        transaction_year, transaction_month, _ = transaction_date.split('-')

        # Check if the transaction matches the selected month and year
        if ((selected_month == "All" or selected_month == transaction_month) and 
            (selected_year == "All" or selected_year == transaction_year)):
            filtered_rows.append(row)

    # Insert filtered data into the Treeview and calculate totals
    for row in filtered_rows:
        tree.insert('', tk.END, values=row)
        amount = row[1]
        category = row[2]

        # Calculate totals based on the category
        if category in ['One Time Expenses', 'Recurrent Expenses']:
            total_expenses += amount
        elif category in ['Short-term Savings', 'Long-term Savings', 'Investment']:
            total_savings += amount

    # Update the labels with the calculated totals
    net_amount = total_savings - total_expenses
    total_expenses_label.config(text=f"Total Expenses: ${total_expenses:.2f}")
    total_savings_label.config(text=f"Total Savings: ${total_savings:.2f}")
    net_amount_label.config(text=f"Net Amount: ${net_amount:.2f}")
        
# Function to manually refresh the transactions displayed
def refresh_transactions_button():
    refresh_transactions()
    messagebox.showinfo("Refreshed", "Transactions have been refreshed.")

        
def clear_entries():
    amount_entry.delete(0, tk.END)
    category_entry.delete(0, tk.END)
    description_entry.delete(0, tk.END)


main_frame = ttk.Frame(root, padding="10")
main_frame.pack(fill=tk.BOTH, expand=True)
main_frame.place(relwidth=1, relheight=1)

ttk.Label(main_frame, text="Date:").place(x=10, y=20)

day_combobox = ttk.Combobox(main_frame, values=list(range(1, 31)))
day_combobox.place(x=120, y=20)
day_combobox.set("Day")

month_combobox = ttk.Combobox(main_frame, values=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"])
month_combobox.place(x=350, y=20)
month_combobox.set("Month")

year_combobox = ttk.Combobox(main_frame, values=list(range(2020, 2100)))
year_combobox.place(x=560, y=20)
year_combobox.set("Year")

# Transaction form fields
ttk.Label(main_frame, text="Amount ($):").place(x=10, y=50)
amount_entry = ttk.Entry(main_frame)
amount_entry.place(x=120, y=50)

ttk.Label(main_frame, text="Category:").place(x=10, y=90)
categories = ['One Time Expenses', 'Recurrent Expenses', 'Short-term Savings', 'Long-term Savings', 'Investment']
category_combobox = ttk.Combobox(main_frame, values=categories)
category_combobox.place(x=120, y=90)
category_combobox.current(0) 

ttk.Label(main_frame, text="Description:").place(x=10, y=130)
description_entry = ttk.Entry(main_frame)
description_entry.place(x=120, y=130)

total_expenses_label = ttk.Label(main_frame, text="Total Expenses: $0")
total_expenses_label.place(x=600, y=50)

total_savings_label = ttk.Label(main_frame, text="Total Savings: $0")
total_savings_label.place(x=600, y=90)

net_amount_label = ttk.Label(main_frame, text="Net Amount: $0")
net_amount_label.place(x=600, y=130)


# Add transaction button
add_button = ttk.Button(main_frame, text="Add Transaction", command=add_transaction)
add_button.place(x=350, y=50)

# Delete transaction button
delete_button = ttk.Button(main_frame, text="Delete Transaction", command=delete_transaction)
delete_button.place(x=350, y=90)

# Edit transaction button
edit_button = ttk.Button(main_frame, text="Edit Transaction", command=edit_transaction)
edit_button.place(x=350, y=130)

# Refresh transactions button
refresh_button = ttk.Button(main_frame, text="Refresh Transactions", command=refresh_transactions_button)
refresh_button.place(x=350, y=170)  # Adjust the position as needed


filter_month_combobox = ttk.Combobox(main_frame, values=["All"] + ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"], width=10)
filter_month_combobox.place(x=20, y=170)
filter_month_combobox.set("All")

filter_year_combobox = ttk.Combobox(main_frame, values=["All"] + list(range(2020, 2031)), width=10)
filter_year_combobox.place(x=200, y=170)
filter_year_combobox.set("All")

filter_button = ttk.Button(main_frame, text="Filter Transactions", command=refresh_transactions)
filter_button.place(x=600, y=170)

# Treeview for displaying transactions
tree = ttk.Treeview(main_frame, columns=('ID', 'Amount', 'Category', 'Date', 'Description'), show='headings')
tree.heading('ID', text='ID')
tree.heading('Amount', text='Amount ($)')
tree.heading('Category', text='Category')
tree.heading('Date', text='Date (yy-mm-dd)')
tree.heading('Description', text='Description')

# Set the column widths
tree.column('ID', width=50)  # Set a smaller width for the 'ID' column
tree.column('Amount', width=100)  # Set a smaller width for the 'Amount' column

tree.place(x=10, y=200, width=780, height=150)  # Adjust size as needed


# Call to create the database and initially populate the Treeview
create_database()
refresh_transactions()

root.mainloop()
